## Useful imports

In [ ]:
# general imports
import os, sys, time, math, re, copy, random, json
from Pathlib import Path

# Live-reload of imported modules
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np

#  to supress warnings
import warnings
warnings.filterwarnings("ignore")

# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# visualization imports
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

%matplotlib inline 

In [ ]:
# ML imports
from sklearn.metrics import classification_report

In [ ]:
#  if using colab
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv("/content/train.csv")

## Logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
RANDOM_STATE = 42

In [ ]:
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')

def setup_logger(name, log_file, level = logging.INFO):
    handler = logging.FileHandler(log_file)
    handler.setFormatter(formatter)
    
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    
    return logger
    
# logger for training accuracies
train_logger = setup_logger('Training_accuracy', '../log/train_output.log')

# logger for evaluation accuracies
eval_logger = setup_logger('Evaluation_accuracy', '../log/eval_output.log')    

In [ ]:
def print_and_log(log_file, message):
    print(message, flush=True)
    log_file.write(message + '\n')

## Seeds

In [ ]:
#############################################
#setting up seeds
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)
########################################################

## Command line parser

In [ ]:
import argparse

def parse_command_line(self):
    parser = argparse.ArgumentParser()
    parser.add_argument("--type_choice", choices=["abc", "f", "g"], default="f", help="Choose <choice>")
    parser.add_argument("--learning_rate", "-lr", type=float, default=0.001, help="Learning rate")
    parser.add_argument("--type_bool", default=False, action="store_true", help="...")

## Date Operations

In [ ]:
from datetime import datetime

# date to desired format
date = datetime.today().strftime('%d-%m-%Y')
date= datetime.strptime(date,'%Y%m%d').strftime('%Y-%m-%d')

# Calender week to date
week= int(date.replace("CW",""))
year = 2021
date = datetime.date(year,1,1)+relativedelta(weeks=+week)  

## Printing markdown through code

In [ ]:
# This function makes it easier to print rendered markdown through a code cell

from IPython.display import Markdown

def mprint(text, *args, **kwargs):
    if 'end' in kwargs.keys():
        text += kwargs['end']

    display(Markdown(text))

In [ ]:
#  Example
with open("data.json", "r", encoding = "utf-8") as f:
    data_dict = json.loads(f)

for k,v in data_dict.items():
    mprint(f"We have **{len(v)}** rows of data in `{k}`.")

In [5]:
# Printing whitespace and aligning with f string

print(f"{'A':<5} {'B':>10}")

x = y = 10

print(f"{x:<5} {y:>10}")

A              B
10            10


### Custom context manager

In [6]:
from contextlib import contextmanager

@contextmanager
def template():
    #  Do everything that will be executed inside the context
    yield
    # Do everything that will be executed right outside the context

{1: 'abc', 2: 'xyz'}

## Sending an Email

In [ ]:
import pandas as pd
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Set up the email addresses and password. Please replace below with your email address and password
email_from = 'bwencs@gmail.com'
password = 'dcefcerf'
email_to = ['xyz@yahoo.com', 'abc@gmail.com']

# Generate today's date to be included in the email Subject
date_str = pd.Timestamp.today().strftime('%Y-%m-%d')

# Define the HTML document
html = '''
    <html>
        <body>
            <h1>Daily BSE Bulk Deals report</h1>
        </body>
    </html>
    '''
# Create a MIMEMultipart class, and set up the From, To, Subject fields
email_message = MIMEMultipart()
# email_message['From'] = email_from
# email_message['To'] = email_to
email_message['Subject'] = f'Report email - {date_str}'
email_message.attach(MIMEText(html, "html"))

# Open the attachment file for reading in binary mode, and make it a MIMEApplication class
with open(filename, "rb") as f:
    file_attachment = MIMEApplication(f.read())
# Add header/name to the attachments    
file_attachment.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)
# Attach the file to the message
email_message.attach(file_attachment)

email_string = email_message.as_string()
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.ehlo()
    # server.starttls()
    server.login(email_from, password)
    server.sendmail(email_from, email_to, email_string)

## Unit Testing

In [ ]:
import os
import sys
import unittest

class Transformer:
    pass

class TestExtractor(unittest.TestCase):
    def test_if_csv_exits(self, csv_path="data.csv"):
        self.assertTrue(os.path.exists(csv_path))

class TestTransformer(unittest.TestCase):
    def test_transformer_valid_unit_mm(self):
        transformer = Transformer(
            raw_data=[
                {
                    "entry": "1",
                    "cake_diameter": "56.78",
                    "diam_unit": "mm",
                    "flavor": "caramel",
                    "is_cake_vegan": "No",
                }
            ]
        )
        res = transformer.transform_data()[0]
        self.assertEqual(res.diameter_in_mm, 56.78)
        self.assertFalse(res.vegan)
        self.assertIsNone(res.is_cake_vegan)
        assert res.flavor=="caramel", "The diameters don't match"

if __name__ == '__main__':
    unittest.main()

## Faster way to read big table with datatable (Have to try)

In [ ]:
%%time

import datatable as dt

dt_df = dt.fread('../path/to/huge/table.csv')
pd_df = dt_df.to_pandas

## Panads tricks

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('data.csv')

# df with only int columns
newdf = df.select_dtypes(include='int64')

#  numeric display precision
pd.options.display.precision = 3

#  format cells
def red_black(val):
    color = "red" if val < 0 else "black"
    return f"color: {color}"
df.style.applymap(red_black)

cmap = sns.light_palette("red", 
                as_cmap=True)
df.style.background_gradient(cmap)

## Pickling

In [ ]:
import pickle
with open('file.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('file.pkl', 'rb') as f:
    data = pickle.load(f)